In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM, Bidirectional

In [3]:
! nvidia-smi

Sun Aug 30 05:31:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH01AWA/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH01AWA/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH01AWA/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [35]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [36]:
tokenizer = info.features['text'].encoder

In [37]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [38]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

In [39]:
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

In [ ]:
# Xây dựng RNN

In [41]:
rnn_model = Sequential()
rnn_model.add(Embedding(tokenizer.vocab_size, 64))
rnn_model.add(SimpleRNN(32))
rnn_model.add(Dense(16, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

In [42]:
rnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 527,489
Trainable params: 527,489
Non-trainable params: 0
_________________________________________________________________


In [43]:
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
rnn_model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
# Xây dựng LSTM

In [51]:
lstm_model = Sequential()
lstm_model.add(Embedding(tokenizer.vocab_size, 64))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(16, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

In [52]:
lstm_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17        
Total params: 536,801
Trainable params: 536,801
Non-trainable params: 0
_________________________________________________________________


In [54]:
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='acc')

In [ ]:
lstm_model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [57]:
# Xây dựng Bi-Directional
bidrect_model = Sequential()
bidrect_model.add(Embedding(tokenizer.vocab_size, 64))
bidrect_model.add(Bidirectional(LSTM(32)))
bidrect_model.add(Dense(16, activation='relu'))
bidrect_model.add(Dense(1, activation='sigmoid'))

In [58]:
bidrect_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                24832     
_________________________________________________________________
dense_11 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 549,729
Trainable params: 549,729
Non-trainable params: 0
_________________________________________________________________


In [62]:
bidrect_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='acc')

In [ ]:
bidrect_model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
    342/Unknown - 35s 102ms/step - loss: 0.5311 - acc: 0.7260